In [ ]:
!pip install datasets
!pip install sentence_transformers
!pip install scikit-learn
!pip install qdrant-client sentence-transformers

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np
from google.colab import userdata
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm import tqdm
import uuid

In [ ]:
ds = load_dataset("ruslanmv/ai-medical-chatbot")

In [ ]:
ds["train"][0]["Description"]

In [ ]:
qa_pairs = [(entry["Patient"], entry["Doctor"]) for entry in ds["train"]]
questions, answers = zip(*qa_pairs)

In [ ]:
# Initialize model with GPU
embed_model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

In [ ]:
# ---- Config ----
QDRANT_HOST = userdata.get("QDRANT_HOST")
QDRANT_API_KEY = userdata.get("QDRANT_API_KEY")
COLLECTION_NAME = "ruslanmv-ai-medical-chatbot"

In [ ]:
# ---- Initialize Qdrant Client ----
client = QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
)

In [ ]:
# ---- Create Collection ----
if not client.collection_exists(collection_name=COLLECTION_NAME):
    print("Creating collection", COLLECTION_NAME)
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=384,  # Depends on your embedding model, all-MiniLM-L6-v2 is a 384 dimensional dense vector space
            distance=models.Distance.COSINE
        )
    )
else:
    print("Collection already exists")

In [ ]:
# Process embeddings in batches
batch_size = 64  # Larger batch size for GPU
question_embeddings = embed_model.encode(
    questions,
    batch_size=batch_size,
    convert_to_numpy=True,
    show_progress_bar=True,
    device="cuda"
)

In [ ]:
# Upload to Qdrant

"""
'answer' is needed in the payload, otherwise, finding the context will be harder and costly.
It is important to have an id that can be identified later, having uuids randomly is not good.
"""

for i in tqdm(range(0, len(questions), batch_size)):
    batch_questions = questions[i : i + batch_size]
    batch_answers = answers[i : i + batch_size]
    batch_vectors = question_embeddings[i : i + batch_size]

    points_batch = [
        models.PointStruct(
            id=i+j+1,
            vector=batch_vectors[j],
            payload={"question": batch_questions[j], "answer": batch_answers[j]}
            )
        for j in range(len(batch_questions))
    ]
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=points_batch,
    )

In [ ]:

nearest = client.query_points(
    collection_name=COLLECTION_NAME,
    query=embed_model.encode("Hi doctor, I fell on the stairs and I hurt my anckle very bad, what should I do?"),
    limit=3
)

for point in nearest.points:
    print(point)

In [ ]:
def retrieve_context(query: str) -> str:
    nearest = client.query_points(
        collection_name=COLLECTION_NAME,
        query=embed_model.encode(query),
        limit=3
    )
    return "\n\n".join([f"Patient: {question}\nDoctor: {answer}" for question, answer in [(point.payload["question"], point.payload["answer"]) for point in nearest.points]])